In [ ]:
# ==============
# CONFIG
# ==============

import astrique_module
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

PREDICTOR1 = 'voicing'                    # first predictor column name
PREDICTOR2 = 'duration'                   # second predictor column name
FILENAME_COL = 'filename'                 # filename column name
LABEL_MAPPING = {'s': 0, 'z': 1}          # binary output label mapping

TARGET = 'answer_batch'                   # target column name
DATA_PATH = 'data/data.csv'               # sound info data file path
PARTICIPANT_CSV_DIR = 'data/participants' # participant CSV directory
PROCESSED_PATH = 'data_processed.csv'     # processed data file path; leave blank to disable

INIT_RANDOM_SAMPLES = 10                  # initial random samples to collect
MIN_ITERATIONS = 30                       # minimum number of iterations
CLEANSER_FREQUENCY = 0                    # insert a high-certainty sample every nth iteration to prevent participant fatigue (irrelevant for virtual agents); 0 to disable
MODEL_CERTAINTY_CUTOFF = 0.95             # stopping certainty threshold
PARTICIPANT_TO_MODEL = 'p03'              # participant ID to simulate

In [ ]:
# ==============
# VIRTUAL AGENT FUNCTIONS
# ==============

def query_participant_classification(filename):
    """
    Queries a virtual agent for a classification of a given sample
    """
    # look into the participant's answer lookup table - PARTICIPANT_CSV_DIR/PARTICIPANT_TO_MODEL.csv
    # return the real class based on LABEL_MAPPING
    participant_answers = pd.read_csv(PARTICIPANT_CSV_DIR + '/' + PARTICIPANT_TO_MODEL + '.csv')
    real_answer = participant_answers[participant_answers[FILENAME_COL] == filename][TARGET].values[0]
    return LABEL_MAPPING[real_answer]

def evaluate_model(stimuli, filename_col):
    """
    Evaluate model predictions on the unanswered data by comparing them to real labels
    obtained via query_participant_classification().
    """
    # get unanswered data
    unanswered = stimuli[stimuli['participant_classification'].isna()].copy()
    
    if unanswered.empty:
        print("No unanswered data to evaluate.")
        return

    # query the actual class for evaluation
    true_labels = []
    predicted_labels = unanswered['predicted_class'].tolist()

    print("Evaluating model predictions on unanswered data...")

    for fname in unanswered[filename_col]:
        true_label = int(query_participant_classification(fname))
        true_labels.append(true_label)

    # calculate metrics
    acc = accuracy_score(true_labels, predicted_labels)
    cm = confusion_matrix(true_labels, predicted_labels)
    report = classification_report(true_labels, predicted_labels)

    print("\n=== Evaluation on Unanswered Data ===")
    print(f"Accuracy: {acc:.4f}")
    print("Confusion Matrix:")
    print(cm)
    print("\nClassification Report:")
    print(report)

def plot_results(stimuli, model):
    """Visualize results with decision boundary and improved legend/color bar"""
    
    # split answered and unanswered data
    answered_data = stimuli[stimuli['participant_classification'].notna()]
    unanswered_data = stimuli[stimuli['participant_classification'].isna()]

    plt.figure(figsize=(10, 6), dpi=300)
    
    # convert answers to numeric if necessary
    if answered_data['participant_classification'].dtype == 'object':
        answered_data = answered_data.copy()
        answered_data['participant_classification'] = answered_data['participant_classification'].astype(int)
        
    # plot answered points, split by class
    for label_char, label_num in LABEL_MAPPING.items():
        subset = answered_data[answered_data['participant_classification'] == label_num]
        if not subset.empty:
            plt.scatter(
                subset[PREDICTOR1],
                subset[PREDICTOR2],
                c='blue' if label_num == 0 else 'red',
                label=f"answered ({label_char})",
                edgecolors='k'
            )

    # plot unanswered points
    if not unanswered_data.empty:
        plt.scatter(
            unanswered_data[PREDICTOR1], 
            unanswered_data[PREDICTOR2],
            c='gray',
            alpha=0.5,
            label='unanswered'
        )
    
    # decision boundary grid
    x_min, x_max = stimuli[PREDICTOR1].min() - 1, stimuli[PREDICTOR1].max() + 1
    y_min, y_max = stimuli[PREDICTOR2].min() - 1, stimuli[PREDICTOR2].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))
    
    grid_points = pd.DataFrame(
        np.c_[xx.ravel(), yy.ravel()],
        columns=[PREDICTOR1, PREDICTOR2]
    )
    
    Z = model.predict_proba(grid_points)[:, 1]
    Z = Z.reshape(xx.shape)
    
    # show background decision gradient
    contour = plt.contourf(xx, yy, Z, alpha=0.3, levels=20, cmap='coolwarm')
    
    # custom color bar with labels s and z
    cbar = plt.colorbar(contour, ticks=[0, 1])
    rev_label_map = {v: k for k, v in LABEL_MAPPING.items()}
    cbar.ax.set_yticklabels([rev_label_map[0], rev_label_map[1]])
    cbar.set_label('predicted answer')
    
    plt.xlabel(PREDICTOR1)
    plt.ylabel(PREDICTOR2)
    plt.title(f'Virtual Agent Results (participant: {PARTICIPANT_TO_MODEL})')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# ==============
# MAIN EXECUTION
# ==============

# create stimuli dataframe
stimuli = pd.read_csv(DATA_PATH)

astrique_module.initialize_dataframe(stimuli)

# initial random sampling with class balance
collected_classes = set()

iteration = 1
while iteration <= INIT_RANDOM_SAMPLES or len(collected_classes) < 2:
    print(f"Iteration {iteration}: Random sampling")

    # select a random stimulus where real class is unknown
    sample = stimuli[stimuli['participant_classification'].isna()].sample(1)

    # get classification, querying filename
    classification = int(query_participant_classification(sample[FILENAME_COL].values[0]))
    
    collected_classes.add(classification)

    # update row in dataframe
    idx = stimuli[FILENAME_COL] == sample[FILENAME_COL].values[0]
    stimuli.loc[idx, 'classification_order'] = iteration
    stimuli.loc[idx, 'classification_type'] = 'random'
    stimuli.loc[idx, 'participant_classification'] = classification

    iteration += 1

# train initial model
model = astrique_module.train_model(stimuli)

# active learning phase
active_learning_iteration = 1

while True:
    # retrain model to get up-to-date predictions on remaining unlabeled samples
    model = astrique_module.train_model(stimuli)

    # get updated unanswered subset
    unanswered = stimuli[stimuli['participant_classification'].isna()]
    
    # check stopping condition
    below_cutoff = unanswered['prediction_certainty'] < MODEL_CERTAINTY_CUTOFF
    if below_cutoff.sum() == 0 and active_learning_iteration >= MIN_ITERATIONS:
        print("Stopping active learning: all predictions above certainty threshold "
              f"({MODEL_CERTAINTY_CUTOFF}) and minimum iterations met ({MIN_ITERATIONS}).")
        break

    # select next sample using uncertainty sampling (with optional cleanser)
    sample, sample_type = astrique_module.get_sample(unanswered, iteration, active_learning_iteration)

    # query real classification
    classification = int(query_participant_classification(sample[FILENAME_COL].values[0]))

    # update row in dataframe
    idx = stimuli[FILENAME_COL] == sample[FILENAME_COL].values[0]
    stimuli.loc[idx, 'classification_order'] = iteration
    stimuli.loc[idx, 'classification_type'] = sample_type
    stimuli.loc[idx, 'participant_classification'] = classification

    iteration += 1
    active_learning_iteration += 1

# evaluate model
evaluate_model(stimuli, FILENAME_COL)

# plot the results
plot_results(stimuli, model)

# save dataframe
if PROCESSED_PATH:
    astrique_module.export_data(stimuli, PROCESSED_PATH)
else:
    print("Processed data not saved - PROCESSED_PATH is empty")
